Le but de ce TP est de mettre en oeuvre le classifieur Bayesien Naif sur les données MNIST. 
Pour se familiariser avec ces données voir [cette page](https://perso.limsi.fr/Individu/allauzen/webpages/pmwiki.php?n=Cours.Main#toc6). Elles sont également accessible en lecture et en local: 

    /partage/public/allauzen
    
Ce corpus rassemble des images (observations) de chiffres isolés et manuscrits ainsi que les classes associées (les labels ou les réponses attendues). Il s'agit donc d'un problème de classification à 10 classes (de 0 à 9). L'objectif est d'assigner le bon chiffre à une image. 

# Chargement et manipulation des données

Reprendre le TP de la semaine dernière afin de charger et manipuler les donner. Vous trouverez ici le strict nécessaire. 

In [20]:
import cPickle, gzip, os, math
import numpy as np
# Load the dataset
f = gzip.open(os.getcwd() + "/mnist.pkl.gz", 'rb')
train_set, valid_set, test_set = cPickle.load(f)

print str(len(train_set[0]))+"\ttraining examples"
print str(len(valid_set[0]))+"\tvalid examples"
print str(len(test_set[0]))+"\ttest examples"

50000	training examples
10000	valid examples
10000	test examples


# Bayesien Naif (continu)
Les observations sont de nature continue. Chaque image contient 784 pixels dont la valeur est réelle et normalisée entre 0 et 1. On pourrait binariser ce type de données, mais ici nous allons les modéliser ainsi en utilisant des gaussiennes.

Une image est un vecteur de caractéristiques contenant 784 composantes. Partons de cette simple représentation. L'hypothèse naïve implique que chaque composante est indépendante des autres. Dans le cas présent, chaque pixel est donc modélisé par une gaussienne. La matrice de covariance est donc diagonale. 

## TODO
Implémenter le Bayesien naïf gaussien: 
- Calcul des paramètres à partir des données d'apprentissage
 - Pour chaque classe calculer le vecteur des moyennes
 - La matrice de covariance (Attention elle est diagonale, on peut se contenter d'un vecteur)
- Inférence sur les données de test
- Calcul du taux d'erreur sur les données de test (en s'y prenant bien, on doit atteindre les 77%)
- Tracer les images "moyennes" pour chaque classe ainsi que les variances
- Regarder quelques images mal classées par le classifieur



In [26]:
# Récupération des images et des labels
images = test_set[0]
labels = test_set[1]
means = dict()
variances = dict()
ids = list()
for i in range(10):
    mesImages = images[labels==i] # Les images de la classe i
    #Calcul moyen
    mean = mesImages.mean(axis=0)
    means[i] = mean
    var = mesImages.var(axis=0)
    variances[i] = var

def calculPost(image):
    """
    Proba a posteriori pour toutes les classes
    """
    post = np.zeros([10, 1])
    for lbl in range(10):
        m = means[lbl]
        v = variances[lbl]
        non_null = v != 0
        s1 = -0.5 * np.log(2 * math.pi * v[non_null])
        s2 = -0.5 * np.divide(np.square(image[non_null] - m[non_null]), v[non_null])
        post[lbl] = (s1 + s2).sum()
    return(post)

posts = list()
good = 0

for i in range(len(images)):
    post = calculPost(images[i])
    if post.argmax() == labels[i]:
        good += 1
print good * 100.0 / len(images)

77.98


# Bayésien naïf discret

Reprendre la même tâche mais en binarizant les images. Pour cela, le modèle de chaque pixel est une binomiale. 

## TODO
- Quels sont les paramètres à estimer pour chaque classe ? 
- Comment les stockés ? 
- Implémenter l'estimation, puis l'inférence. (On peut dans un premier temps réutiliser l'implémentation existante de Sklearn)
